In [20]:
import pathlib
import datetime
import json

DATA_DIR = pathlib.Path('/home/michel/projects/desinfo/political-ads-scraper/data')

In [15]:
ads_by_date = {}

for filename in sorted(list((DATA_DIR / 'twitter').iterdir())):
    fetch_datetime = datetime.datetime.strptime(
        filename.stem,
        'twitter-ads_EU_%Y-%m-%d_%H-%M-%S',
    )

    with open(filename, 'r') as f:
        ads = json.load(f)

    ads_by_date[fetch_datetime.strftime("%Y-%m-%d")] = ads


In [19]:
ads_by_date.keys()

dict_keys(['2019-06-07', '2019-06-08', '2019-06-09', '2019-06-10', '2019-06-11', '2019-06-12', '2019-06-13', '2019-06-14'])

In [31]:
ads_by_date['2019-06-10'].keys()

dict_keys(['advertisers_data', 'advertisers_tweets'])

## Advertisers

In [35]:
adv_lists = [
    tuple([
        adv['id_str']
        for adv in ads_by_date[date]['advertisers_data']
    ])
    for date in ads_by_date.keys()
]

In [55]:
assert len(set(adv_lists)) == 1
adv_ids = adv_lists[0]

## Tweets

In [64]:
for adv_id in adv_ids:
    for date_str in list(ads_by_date.keys())[:-1]:
        date = datetime.date(*map(int, date_str.split('-')))
        date_tomorrow = date + datetime.timedelta(1)

        tweet_ids_1 = set([
            tweet['basic_data']['tweetId']
            for tweet in ads_by_date[date_str]['advertisers_tweets'][adv_id]
        ])
        tweet_ids_2 = set([
            tweet['basic_data']['tweetId']
            for tweet in ads_by_date[str(date_tomorrow)]['advertisers_tweets'][adv_id]
        ])

        if not (tweet_ids_1 <= tweet_ids_2):
            print(adv_id, date_str, tweet_ids_1, tweet_ids_2)


57043880 2019-06-10 {'1135551352338362372', '1135579175899865088', '1135835273353474049', '1135846176903876608', '1135817285191569408', '1135888800746487808', '1135592780724674560', '1135878229519228929', '1135598139535233024', '1135798285158014976', '1133010619328860161', '1064824170196819968', '1135595001667305472', '1135856470761889792', '1135586726435532800', '1135877625807851520', '1135859860308930560', '1135856867320766465', '1135594439374774285', '1135854305586634752', '1135883995227533313', '1135845438920232960', '1135822085304061954', '1135826961245323264', '1135824292204167176', '1135815219945648128', '1135816966734786560', '1135831458378125312', '1135602040682532865', '1135891028081303552', '1131467518923685888', '1135827478008750082', '1064823921306812416', '1135824351561945088', '1135561706371989505', '1136262049502244865', '1135570464661745665', '1135555016574537729', '1135582207098130432', '1135856805123383296', '1135887120558632961', '1064823918492405761', '113587782150

In [66]:
adv_id = '57043880'
for date_str in ads_by_date.keys():
    date = datetime.date(*map(int, date_str.split('-')))

    tweet_ids = set([
        tweet['basic_data']['tweetId']
        for tweet in ads_by_date[date_str]['advertisers_tweets'][adv_id]
    ])

    print(date, len(tweet_ids))


2019-06-07 107
2019-06-08 107
2019-06-09 107
2019-06-10 107
2019-06-11 15
2019-06-12 15
2019-06-13 15
2019-06-14 15
